# It definitely came as a surprise to me that there was this huge shake up in the end. 
I personally thought that the shake up should be relatively small because my stacking 
model performance on public LB and local cv was always aligned although for single model 
they could vary quite a bit. We did trust mostly on local CV and we tried very hard on nonlinear 
stacking (without success) so we relied on weighted average cv score and only submitted when there was an improvement on local cv.

# I cannot wait to see what @Michael Jahrer did but I would like to share some of our approaches (actually, just the NN part, and leave the rest to my teammates) before Michael publishes his and then nobody cares what we did :(

# So my best NN had a 5 fold local CV 0.294, and public LB 0.284 and private 0.290. And here is roughly how you can achieve that:

# 1) important features' interactions (e.g. ps_car_13, ps_ind_03, ps_reg_03, ...)

# 2) count of categorical features

# 3) xgboost predictions: divide feature sets into three groups (car, ind, reg) and then use two group as features and the other group as target, train a xgboost model on it, and use prediction as features

# 4) feature aggregation: pick two features (e.g. ps_car_13, ps_ind_03), and then use one as group variable, the other as value variable, do mean, std, max, min, median. Still top important features are picked only

# 5) do embedding layers on all categorical features (I used embedding dimension 4 with dropout 0.25)

# 6) model is 2 layer with relu activation and big dropout (512 with dropout 0.8, and 64 with 0.8)

# 7) a few categorical count features @qianqian created

# That is about it! Hope it helps :)

# Update: my best lightgbm code is here

# https://www.kaggle.com/xiaozhouwang/2nd-place-lightgbm-solution

# It scored 0.29124 on private LB.

# best nn code is here https://www.kaggle.com/xiaozhouwang/2nd-place-solution-nn-model

# It scored 0.29008 on private LB.

# And their model results can be found in the comments (Seems I cannot upload files here?)

In [1]:
# part of 2nd place solution: lightgbm model with private score 0.29124 and public lb score 0.28555

import lightgbm as lgbm
from scipy import sparse as ssp
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def Gini(y_true, y_pred):
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]

    # sort rows on prediction column
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:, 0].argsort()][::-1, 0]
    pred_order = arr[arr[:, 1].argsort()][::-1, 0]

    # get Lorenz curves
    L_true = np.cumsum(true_order) * 1. / np.sum(true_order)
    L_pred = np.cumsum(pred_order) * 1. / np.sum(pred_order)
    L_ones = np.linspace(1 / n_samples, 1, n_samples)

    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)

    # normalize to true Gini coefficient
    return G_pred * 1. / G_true

cv_only = True
save_cv = True
full_train = False

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', Gini(labels, preds), True

path = "../data/"

train = pd.read_csv(path+'train.csv')
train_label = train['target']
train_id = train['id']
test = pd.read_csv(path+'test.csv')
test_id = test['id']

NFOLDS = 5
kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=218)

y = train['target'].values
drop_feature = [
    'id',
    'target'
]

X = train.drop(drop_feature,axis=1)
feature_names = X.columns.tolist()
cat_features = [c for c in feature_names if ('cat' in c and 'count' not in c)]
num_features = [c for c in feature_names if ('cat' not in c and 'calc' not in c)]

train['missing'] = (train==-1).sum(axis=1).astype(float)
test['missing'] = (test==-1).sum(axis=1).astype(float)
num_features.append('missing')

for c in cat_features:
    le = LabelEncoder()
    le.fit(train[c])
    train[c] = le.transform(train[c])
    test[c] = le.transform(test[c])

enc = OneHotEncoder()
enc.fit(train[cat_features])
X_cat = enc.transform(train[cat_features])
X_t_cat = enc.transform(test[cat_features])

ind_features = [c for c in feature_names if 'ind' in c]
count=0
for c in ind_features:
    if count==0:
        train['new_ind'] = train[c].astype(str)+'_'
        test['new_ind'] = test[c].astype(str)+'_'
        count+=1
    else:
        train['new_ind'] += train[c].astype(str)+'_'
        test['new_ind'] += test[c].astype(str)+'_'

cat_count_features = []
for c in cat_features+['new_ind']:
    d = pd.concat([train[c],test[c]]).value_counts().to_dict()
    train['%s_count'%c] = train[c].apply(lambda x:d.get(x,0))
    test['%s_count'%c] = test[c].apply(lambda x:d.get(x,0))
    cat_count_features.append('%s_count'%c)

train_list = [train[num_features+cat_count_features].values,X_cat,]
test_list = [test[num_features+cat_count_features].values,X_t_cat,]

X = ssp.hstack(train_list).tocsr()
X_test = ssp.hstack(test_list).tocsr()


# learning_rate = 0.1
# num_leaves = 15
# min_data_in_leaf = 2000
# feature_fraction = 0.6
# num_boost_round = 10000
# params = {"objective": "binary",
#           "boosting_type": "gbdt",
#           "learning_rate": learning_rate,
#           "num_leaves": num_leaves,
#            "max_bin": 256,
#           "feature_fraction": feature_fraction,
#           "verbosity": 0,
#           "drop_rate": 0.1,
#           "is_unbalance": False,
#           "max_drop": 50,
#           "min_child_samples": 10,
#           "min_child_weight": 150,
#           "min_split_gain": 0,
#           "subsample": 0.9
#           }

# x_score = []
# final_cv_train = np.zeros(len(train_label))
# final_cv_pred = np.zeros(len(test_id))
# for s in range(16):
#     cv_train = np.zeros(len(train_label))
#     cv_pred = np.zeros(len(test_id))

#     params['seed'] = s

#     if cv_only:
#         kf = kfold.split(X, train_label)

#         best_trees = []
#         fold_scores = []

#         for i, (train_fold, validate) in enumerate(kf):
#             X_train, X_validate, label_train, label_validate = \
#                 X[train_fold, :], X[validate, :], train_label[train_fold], train_label[validate]
#             dtrain = lgbm.Dataset(X_train, label_train)
#             dvalid = lgbm.Dataset(X_validate, label_validate, reference=dtrain)
#             bst = lgbm.train(params, dtrain, num_boost_round, valid_sets=dvalid, feval=evalerror, verbose_eval=100,
#                             early_stopping_rounds=100)
#             best_trees.append(bst.best_iteration)
#             cv_pred += bst.predict(X_test, num_iteration=bst.best_iteration)
#             cv_train[validate] += bst.predict(X_validate)

#             score = Gini(label_validate, cv_train[validate])
#             print(score)
#             fold_scores.append(score)

#         cv_pred /= NFOLDS
#         final_cv_train += cv_train
#         final_cv_pred += cv_pred

#         print("cv score:")
#         print(Gini(train_label, cv_train))
#         print("current score:", Gini(train_label, final_cv_train / (s + 1.)), s+1)
#         print(fold_scores)
#         print(best_trees, np.mean(best_trees))

#         x_score.append(Gini(train_label, cv_train))

# print(x_score)
# pd.DataFrame({'id': test_id, 'target': final_cv_pred / 16.}).to_csv('../cache/lgbm3_pred_avg.csv', index=False)
# pd.DataFrame({'id': train_id, 'target': final_cv_train / 16.}).to_csv('../cache/lgbm3_cv_avg.csv', index=False)

In [2]:
from numba import jit
@jit
def eval_gini(y_true, y_prob):
    """
    Original author CPMP : https://www.kaggle.com/cpmpml
    In kernel : https://www.kaggle.com/cpmpml/extremely-fast-gini-computation
    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

def gini_xgb(preds, dtrn_df):
    labels = dtrn_df.get_label()
    gini_score = eval_gini(labels, preds)
    return [('gini', gini_score)]


def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))



In [3]:
X

<595212x223 sparse matrix of type '<class 'numpy.float64'>'
	with 25058414 stored elements in Compressed Sparse Row format>

In [4]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
import gc

import time 
from datetime import datetime

In [5]:
target = train_label
trn_df = pd.DataFrame(X.toarray())
sub_df = pd.DataFrame(X_test.toarray())

n_splits = 5
n_estimators = 200


np.random.seed(0)
final_cv_pred = np.zeros(len(test_id))
x_score = []
for s in range(16):
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=15) 
    imp_df = np.zeros((len(trn_df.columns), n_splits))
    xgb_evals = np.zeros((n_estimators, n_splits))
    oof = np.empty(len(trn_df))
    sub_preds = np.zeros(len(sub_df))
    increase = True
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(target, target)):
        trn_dat, trn_tgt = trn_df.iloc[trn_idx], target.iloc[trn_idx]
        val_dat, val_tgt = trn_df.iloc[val_idx], target.iloc[val_idx]


        # Upsample during cross validation to avoid having the same samples
        # in both trn_df and validation sets
        # Validation set is not up-sampled to monitor overfitting
        if increase:
            # Get positive examples
            pos = pd.Series(trn_tgt == 1)
            # Add positive examples
            trn_dat = pd.concat([trn_dat, trn_dat.loc[pos]], axis=0)
            trn_tgt = pd.concat([trn_tgt, trn_tgt.loc[pos]], axis=0)
            # Shuffle data
            idx = np.arange(len(trn_dat))
            np.random.shuffle(idx)
            trn_dat = trn_dat.iloc[idx]
            trn_tgt = trn_tgt.iloc[idx]

    #     print(sum(trn_tgt==0)/sum(trn_tgt==1))

    #     sp = sum(trn_tgt==0)/sum(trn_tgt==1)
    #         sp = 1.55556
        sp = 1.52631578947
        clf = XGBClassifier(n_estimators=n_estimators,
                            max_depth=4,
                            objective="binary:logistic",
                            learning_rate=.1, 
                            subsample=.8, 
                            colsample_bytree=.8,
                            scale_pos_weight=sp,
                            missing=-1,
                            gamma=1,
                            reg_alpha=0,
                            reg_lambda=1,
                            seed=s,
                            nthread=5)

        clf.fit(trn_dat, trn_tgt, 
                eval_set=[(trn_dat, trn_tgt), (val_dat, val_tgt)],
                eval_metric=gini_xgb,
                early_stopping_rounds=None,
                verbose=False)

        # Keep feature importances
        imp_df[:, fold_] = clf.feature_importances_

        # Find best round for validation set
        xgb_evals[:, fold_] = clf.evals_result_["validation_1"]["gini"]
        # Xgboost provides best round starting from 0 so it has to be incremented
        best_round = np.argsort(xgb_evals[:, fold_])[::-1][0]
        print(best_round)

        # Predict OOF and submission probas with the best round
        oof[val_idx] = clf.predict_proba(val_dat, ntree_limit=int(best_round))[:, 1]
        # Update submission
        sub_preds += clf.predict_proba(sub_df, ntree_limit=int(best_round))[:, 1] / n_splits

        # Display results
        print("Fold %2d : %.6f @%4d / best score is %.6f @%4d"
              % (fold_ + 1,
                 eval_gini(val_tgt, oof[val_idx]),
                 n_estimators,
                 xgb_evals[best_round, fold_],
                 best_round))

    print(sp)
    full_oof_score = eval_gini(target, oof)
    print("Full OOF score : %.6f" % eval_gini(target, oof))
    # Compute mean score and std
    mean_eval = np.mean(xgb_evals, axis=1)
    std_eval = np.std(xgb_evals, axis=1)
    best_round = np.argsort(mean_eval)[::-1][0]

    print("Best mean score : %.6f + %.6f @%4d"
          % (mean_eval[best_round], std_eval[best_round], best_round))
    print('--------------------------------------')
    print('\n')
    
    final_cv_pred += sub_preds
    
    x_score.append(full_oof_score)
print(x_score)
pd.DataFrame({'id': test_id, 'target': final_cv_pred / 16.}).to_csv('../submissions/xgb3c1_pred_avg.csv', index=False)

198
Fold  1 : 0.276865 @ 200 / best score is 0.277186 @ 198
198
Fold  2 : 0.278185 @ 200 / best score is 0.278352 @ 198
182
Fold  3 : 0.303803 @ 200 / best score is 0.303975 @ 182
196
Fold  4 : 0.277825 @ 200 / best score is 0.277850 @ 196
194
Fold  5 : 0.281765 @ 200 / best score is 0.281824 @ 194
1.52631578947
Full OOF score : 0.283610
Best mean score : 0.283682 + 0.010069 @ 198
--------------------------------------


194
Fold  1 : 0.275794 @ 200 / best score is 0.275887 @ 194
188
Fold  2 : 0.278177 @ 200 / best score is 0.278192 @ 188
178
Fold  3 : 0.304004 @ 200 / best score is 0.304217 @ 178
140
Fold  4 : 0.278593 @ 200 / best score is 0.278730 @ 140
186
Fold  5 : 0.279349 @ 200 / best score is 0.279493 @ 186
1.52631578947
Full OOF score : 0.283116
Best mean score : 0.283002 + 0.010554 @ 188
--------------------------------------


188
Fold  1 : 0.274965 @ 200 / best score is 0.275212 @ 188
193
Fold  2 : 0.279591 @ 200 / best score is 0.279668 @ 193
195
Fold  3 : 0.304909 @ 200 /

In [6]:
sum([0.2836097928832232, 0.2831163809243863, 0.28304775908729873, 0.28223102013143964, 0.28189330224497844, 0.28321689917731097, 0.2826142365368659, 0.2820093354922273, 0.28183180922291284, 0.28199787470200854, 0.28296732217325327, 0.2836106041741363, 0.2825641586528257, 0.2837473308854471, 0.28275264032328995, 0.2819471570442742])/16


0.28269735147849245

In [7]:
np.max(final_cv_pred/16)

0.74224331695586443